# Dataset 4 - Boston

## Import Libraries 

In [1]:
import warnings
import sys
sys.path.insert(0, '../src')
warnings.filterwarnings('ignore')

In [2]:
import train_test
import ABC_train_test
import bostonDataset
import baselineModels
import network
import statsModel
import performanceMetrics
import dataset
import sanityChecks
import torch 
import matplotlib.pyplot as plt 
import seaborn as sns 
from torch.utils.data import random_split 

## Parameters

General Parameters 

1. Number of Samples
2. Number of features 

ABC-Generator parameters are as mentioned below: 
1. mean : 1 ($\beta \sim N(\beta^{*},\sigma)$ where $\beta^{*}$ are coefficients of statistical model) or 1 ($\beta \sim N(0,\sigma)$
2. std : $\sigma = 1, 0.1 , 0.01$ (standard deviation)

In [3]:
n_features = 13
n_samples= 506
n_target = 1 
 
#ABC Generator Parameters
mean = 1 
variance = 0.01

#Hyper-parameters 
n_epochs = 100

## Dataset

In [4]:
X,Y = bostonDataset.boston_data()

        X1    X2    X3   X4     X5     X6    X7      X8   X9    X10   X11  \
0  0.00632  18.0  2.31  0.0  0.538  6.575  65.2  4.0900  1.0  296.0  15.3   
1  0.02731   0.0  7.07  0.0  0.469  6.421  78.9  4.9671  2.0  242.0  17.8   
2  0.02729   0.0  7.07  0.0  0.469  7.185  61.1  4.9671  2.0  242.0  17.8   
3  0.03237   0.0  2.18  0.0  0.458  6.998  45.8  6.0622  3.0  222.0  18.7   
4  0.06905   0.0  2.18  0.0  0.458  7.147  54.2  6.0622  3.0  222.0  18.7   

      X12   X13     Y  
0  396.90  4.98  24.0  
1  396.90  9.14  21.6  
2  392.83  4.03  34.7  
3  394.63  2.94  33.4  
4  396.90  5.33  36.2  


## Stats Model

In [ ]:
[coeff,y_pred] = statsModel.statsModel(X,Y)

## Common Training Parameters (GAN & ABC_GAN) 

In [5]:
threshold_mse = 0.99
batch_size = 100
#Select the device 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
# Train test split for dataset 
real_dataset = dataset.CustomDataset(X,Y)
train_size = round(0.8 * n_samples)
test_size = n_samples - train_size
train_data, test_data = random_split(real_dataset,[train_size,test_size])

## Vanilla Neural Network 

In [7]:
baselineModels.vanillaNeuralNetwork(train_data,test_data,batch_size,n_features,n_target,n_epochs)

epoch 1 loss: 0.5104
epoch 2 loss: 1.6939
epoch 3 loss: 0.8445
epoch 4 loss: 0.2373
epoch 5 loss: 0.3755
epoch 6 loss: 0.4627
epoch 7 loss: 1.9223
epoch 8 loss: 0.6038
epoch 9 loss: 2.1714
epoch 10 loss: 0.9472
epoch 11 loss: 0.0772
epoch 12 loss: 0.5998
epoch 13 loss: 0.4626
epoch 14 loss: 2.2960
epoch 15 loss: 0.3220
epoch 16 loss: 0.6537
epoch 17 loss: 0.4494
epoch 18 loss: 0.6778
epoch 19 loss: 0.4014
epoch 20 loss: 0.2795
epoch 21 loss: 2.1886
epoch 22 loss: 2.2009
epoch 23 loss: 2.5967
epoch 24 loss: 0.4447
epoch 25 loss: 0.1773
epoch 26 loss: 0.7010
epoch 27 loss: 0.8926
epoch 28 loss: 3.9688
epoch 29 loss: 0.3768
epoch 30 loss: 2.4123
epoch 31 loss: 3.7445
epoch 32 loss: 0.0429
epoch 33 loss: 2.4505
epoch 34 loss: 0.2939
epoch 35 loss: 0.1971
epoch 36 loss: 0.8316
epoch 37 loss: 1.8365
epoch 38 loss: 0.3378
epoch 39 loss: 0.3952
epoch 40 loss: 2.3054
epoch 41 loss: 1.5890
epoch 42 loss: 0.3121
epoch 43 loss: 1.8182
epoch 44 loss: 0.1401
epoch 45 loss: 2.0911
epoch 46 loss: 2.36

## GAN Model

**Training GAN for n_epochs number of epochs** 

In [8]:
generator = network.Generator(n_features+1).to(device)
discriminator = network.Discriminator(n_features+1).to(device)

criterion = torch.nn.BCELoss()
gen_opt = torch.optim.Adam(generator.parameters(), lr=0.01, betas=(0.5, 0.999))
disc_opt = torch.optim.Adam(discriminator.parameters(), lr=0.01, betas=(0.5, 0.999))

In [9]:
print(generator)
print(discriminator)

Generator(
  (hidden1): Linear(in_features=14, out_features=100, bias=True)
  (hidden2): Linear(in_features=100, out_features=100, bias=True)
  (output): Linear(in_features=100, out_features=1, bias=True)
  (relu): ReLU()
)
Discriminator(
  (hidden1): Linear(in_features=14, out_features=25, bias=True)
  (hidden2): Linear(in_features=25, out_features=50, bias=True)
  (output): Linear(in_features=50, out_features=1, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
)


In [ ]:
discLossG1,genLossG1 = train_test.training_GAN(discriminator,generator,disc_opt,gen_opt,train_data,batch_size, n_epochs,criterion,device)

In [ ]:
GAN1_metrics = train_test.test_generator(generator,test_data,device)

In [ ]:
sanityChecks.discProbVsError(real_dataset,discriminator,device)

**Training GAN until mse of y_pred is > baseline_mse or n_epochs < 5000**

In [ ]:
generator2 = network.Generator(n_features+1).to(device)
discriminator2 = network.Discriminator(n_features+1).to(device)
criterion = torch.nn.BCELoss()
gen_opt = torch.optim.Adam(generator2.parameters(), lr=0.01, betas=(0.5, 0.999))
disc_opt = torch.optim.Adam(discriminator2.parameters(), lr=0.01, betas=(0.5, 0.999))

In [ ]:
discLossG2,genLossG2 = train_test.training_GAN_2(discriminator2,generator2,disc_opt,gen_opt,train_data,test_data,batch_size,threshold_mse,criterion,device)

In [ ]:
GAN2_metrics=train_test.test_generator_2(generator2,test_data,device)

In [ ]:
sanityChecks.discProbVsError(real_dataset,discriminator2,device)

# ABC GAN Model

### Training the network
**Training ABC-GAN for n_epochs number of epochs** 

In [ ]:
gen = network.Generator(n_features+1).to(device)
disc = network.Discriminator(n_features+1).to(device)

criterion = torch.nn.BCELoss()
gen_opt = torch.optim.Adam(gen.parameters(), lr=0.01, betas=(0.5, 0.999))
disc_opt = torch.optim.Adam(disc.parameters(), lr=0.01, betas=(0.5, 0.999))

In [ ]:
discLossA1,genLossA1 = ABC_train_test.training_GAN(disc, gen,disc_opt,gen_opt,train_data,batch_size, n_epochs,criterion,coeff,mean,variance,device)

In [ ]:
ABC_GAN1_metrics=ABC_train_test.test_generator(gen,test_data,coeff,mean,variance,device)

#### Sanity Checks 

In [ ]:
sanityChecks.discProbVsError(real_dataset,disc,device)

**Training ABC-GAN until mse of y_pred is > baseline_mse or n_epochs < 5000**

In [ ]:
gen2 = network.Generator(n_features+1).to(device)
disc2 = network.Discriminator(n_features+1).to(device)

criterion = torch.nn.BCELoss()
gen_opt = torch.optim.Adam(gen2.parameters(), lr=0.01, betas=(0.5, 0.999))
disc_opt = torch.optim.Adam(disc2.parameters(), lr=0.01, betas=(0.5, 0.999))

In [ ]:
discLossA2,genLossA2 = ABC_train_test.training_GAN_2(disc2,gen2,disc_opt,gen_opt,train_data,test_data,batch_size,threshold_mse,criterion,coeff,mean,variance,device)

In [ ]:
ABC_GAN2_metrics=ABC_train_test.test_generator_2(gen2,test_data,coeff,mean,variance,device)

In [ ]:
sanityChecks.discProbVsError(real_dataset,disc2,device)

# Model Analysis

In [ ]:
performanceMetrics.modelAnalysis(GAN1_metrics,ABC_GAN1_metrics,GAN2_metrics,ABC_GAN2_metrics)

In [ ]:
performanceMetrics.plotTrainingLoss(discLossG1,genLossG1,discLossA1,genLossA1,discLossG2,genLossG2,discLossA2,genLossA2)

## GAN Model with skip connection

In [ ]:
generator3 = network.GeneratorWithSkipConnection(n_features+1).to(device)
discriminator3 = network.Discriminator(n_features+1).to(device)

criterion = torch.nn.BCELoss()
gen_opt = torch.optim.Adam(generator3.parameters(), lr=0.01, betas=(0.5, 0.999))
disc_opt = torch.optim.Adam(discriminator3.parameters(), lr=0.01, betas=(0.5, 0.999))

In [ ]:
discLossG3,genLossG3 = train_test.training_GAN(discriminator3,generator3 ,disc_opt,gen_opt,train_data,batch_size, n_epochs,criterion,device)

In [ ]:
GAN3_metrics=ABC_train_test.test_generator(generator3,test_data,coeff,mean,variance,device)

In [ ]:
sanityChecks.discProbVsError(real_dataset,discriminator3,device)

## ABC - GAN Model with skip connection

In [ ]:
gen3 = network.GeneratorWithSkipConnection(n_features+1).to(device)
disc3 = network.Discriminator(n_features+1).to(device)

criterion = torch.nn.BCELoss()
gen_opt = torch.optim.Adam(gen3.parameters(), lr=0.01, betas=(0.5, 0.999))
disc_opt = torch.optim.Adam(disc3.parameters(), lr=0.01, betas=(0.5, 0.999))

In [ ]:
discLossA3,genLossA3 = ABC_train_test.training_GAN(disc3, gen3,disc_opt,gen_opt,train_data,batch_size, n_epochs,criterion,coeff,mean,variance,device)

In [ ]:
ABC_GAN3_metrics=ABC_train_test.test_generator(gen3,test_data,coeff,mean,variance,device)

In [ ]:
sanityChecks.discProbVsError(real_dataset,disc3,device)

In [ ]:
## Skip Connection Model Analysis - GAN and ABC-GAN 

In [ ]:
### Weight Analysis 

##Study the weights of the skip connection layer

In [ ]:
print("GAN Weights")
for name,param in generator3.named_parameters():
    if(name == "skipNode.weight"):
        print(param)
print("ABC-GAN Weights")
for name,param in gen3.named_parameters():
    if(name == "skipNode.weight"):
        print(param)

In [ ]:
performanceMetrics.modelAnalysis2(GAN3_metrics,ABC_GAN3_metrics)

In [ ]:
performanceMetrics.plotTrainingLoss2(discLossG3,genLossG3,discLossA3,genLossA3)